# Day18_1: BERT 파인튜닝

## 학습 목표

**Part 1: 기초**
1. BERT 개념과 아키텍처 이해하기
2. 사전학습 방식(MLM, NSP) 이해하기
3. Hugging Face Transformers 라이브러리 사용하기
4. 토크나이저(Tokenizer) 활용하기
5. 사전학습 모델 로드하기

**Part 2: 심화**
1. 다운스트림 태스크 이해하기
2. Fine-tuning 전략 이해하기
3. 한국어 BERT (multilingual) 활용하기
4. NSMC 한국어 감성 분석 실습하기

---

## 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| BERT | 범용 언어 이해 모델 | 문서 분류, QA, 개체명 인식 |
| 사전학습 | 대규모 데이터로 언어 이해 | Wikipedia, 뉴스 코퍼스 |
| Fine-tuning | 특정 태스크 적응 | 감성 분석, 스팸 탐지 |
| Hugging Face | 쉬운 모델 활용 | 3줄 코드로 최신 모델 사용 |

**분석가 관점**: BERT는 NLP의 ImageNet 모멘트를 가져온 혁명적인 모델입니다. Hugging Face 라이브러리를 사용하면 수천 개의 사전학습 모델을 단 몇 줄의 코드로 활용할 수 있어, 텍스트 분류, 감성 분석, 질의응답 등 다양한 NLP 태스크를 빠르게 해결할 수 있습니다!

---

# Part 1: 기초

---

## 1.1 BERT란?

### BERT (Bidirectional Encoder Representations from Transformers)

2018년 구글이 발표한 **양방향 사전학습 언어 모델**입니다.

```
BERT의 핵심 특징:
1. Bidirectional (양방향): 문맥의 좌우를 모두 고려
2. Transformer Encoder: Self-Attention 기반 구조
3. Pre-training + Fine-tuning: 사전학습 후 미세조정 패러다임
```

### 기존 모델과의 차이점

| 모델 | 방향성 | 문맥 이해 |
|------|--------|----------|
| GPT | 단방향 (좌 -> 우) | 과거 문맥만 참조 |
| ELMo | 양방향 (좌->우 + 우->좌) | 별도 학습 후 결합 |
| **BERT** | **진정한 양방향** | **동시에 좌우 문맥 참조** |

In [ ]:
# 필수 라이브러리 설치 (Colab 또는 새 환경에서 실행)
# !pip install transformers datasets evaluate accelerate torch

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print(f"PyTorch 버전: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

In [ ]:
# 양방향 문맥의 중요성 예시
example1 = "나는 [MASK]에 가서 돈을 찾았다."
example2 = "물고기가 [MASK]에서 헤엄치고 있다."

print("양방향 문맥의 중요성:")
print(f"\n예시 1: {example1}")
print("  -> '돈을 찾았다'를 보면 '은행'(금융)임을 알 수 있음")
print(f"\n예시 2: {example2}")
print("  -> '물고기', '헤엄치고'를 보면 '강/바다'임을 알 수 있음")
print("\n단방향 모델은 [MASK] 이후의 정보를 사용할 수 없습니다!")

### BERT 아키텍처

```
BERT-base:  12 Layers, 768 Hidden, 12 Heads, 110M Parameters
BERT-large: 24 Layers, 1024 Hidden, 16 Heads, 340M Parameters

입력 구조:
[CLS] 토큰1 토큰2 ... 토큰N [SEP] (문장1)
[CLS] 토큰1 토큰2 [SEP] 토큰A 토큰B [SEP] (문장 쌍)

특수 토큰:
- [CLS]: Classification 토큰, 문장 전체 표현
- [SEP]: Separator 토큰, 문장 구분
- [PAD]: Padding 토큰, 길이 맞추기
- [MASK]: Masking 토큰, MLM 학습용
```

### BERT 아키텍처 구조

```mermaid
flowchart TB
    Input["Input Tokens<br/>[CLS] token1 token2 ... [SEP]"] --> Embed["Token Embedding<br/>+ Position Embedding<br/>+ Segment Embedding"]
    Embed --> Encoder1["Encoder Layer 1<br/>Multi-Head Attention<br/>+ Feed Forward"]
    Encoder1 --> Encoder2["Encoder Layer 2"]
    Encoder2 --> EncoderN["Encoder Layer N<br/>(12 for base, 24 for large)"]
    EncoderN --> Outputs["Outputs<br/>(batch, seq_len, hidden_size)"]
    Outputs --> CLS["[CLS] Token Output<br/>(batch, hidden_size)"]
    Outputs --> TokenOutputs["Token Outputs<br/>(batch, seq_len, hidden_size)"]
    
    style Input fill:#ffffff,color:#000000
    style Embed fill:#ffffff,color:#000000
    style Encoder1 fill:#ffffff,color:#000000
    style Encoder2 fill:#ffffff,color:#000000
    style EncoderN fill:#ffffff,color:#000000
    style Outputs fill:#ffffff,color:#000000
    style CLS fill:#ffffff,color:#000000
    style TokenOutputs fill:#ffffff,color:#000000
```

---

## 1.2 사전학습 방식 (Pre-training)

### BERT의 두 가지 사전학습 과제

BERT는 레이블 없는 대규모 텍스트(Wikipedia + BookCorpus)로 자기지도 학습합니다.

### 1) MLM (Masked Language Model)

입력 토큰의 15%를 마스킹하고, 원래 토큰을 예측하는 과제

```
원본: 나는 오늘 학교에 갔다
마스킹: 나는 오늘 [MASK]에 갔다
예측: 학교

마스킹 전략 (15% 토큰 중):
- 80%: [MASK]로 변환
- 10%: 랜덤 토큰으로 변환
- 10%: 원래 토큰 유지
```

In [ ]:
# MLM 예시 시뮬레이션
def simulate_mlm(sentence, mask_prob=0.15):
    """MLM 마스킹 시뮬레이션"""
    tokens = sentence.split()
    masked_tokens = []
    labels = []
    
    for token in tokens:
        if np.random.random() < mask_prob:
            labels.append(token)
            r = np.random.random()
            if r < 0.8:  # 80%: [MASK]
                masked_tokens.append('[MASK]')
            elif r < 0.9:  # 10%: 랜덤 토큰
                masked_tokens.append('[RANDOM]')
            else:  # 10%: 원래 토큰
                masked_tokens.append(token)
        else:
            masked_tokens.append(token)
            labels.append('-')
    
    return masked_tokens, labels

# 시뮬레이션
np.random.seed(42)
sentence = "BERT는 양방향 트랜스포머 인코더로 문맥을 이해합니다"
masked, labels = simulate_mlm(sentence, mask_prob=0.3)  # 시연용으로 30%

print("MLM (Masked Language Model) 예시:")
print(f"원본: {sentence}")
print(f"마스킹: {' '.join(masked)}")
print(f"레이블: {' '.join(labels)}")

### 2) NSP (Next Sentence Prediction)

두 문장이 연속된 문장인지 예측하는 이진 분류 과제

```
[IsNext 예시]
문장 A: 오늘 날씨가 좋다.
문장 B: 공원에 산책을 갔다.
-> IsNext (연속된 문장)

[NotNext 예시]
문장 A: 오늘 날씨가 좋다.
문장 B: 주식 시장이 폭락했다.
-> NotNext (관련 없는 문장)
```

In [ ]:
# NSP 예시
nsp_examples = [
    {
        "A": "서울에서 부산까지 KTX로 2시간 30분 걸린다.",
        "B": "고속철도 덕분에 당일치기 여행이 가능해졌다.",
        "label": "IsNext"
    },
    {
        "A": "서울에서 부산까지 KTX로 2시간 30분 걸린다.",
        "B": "파이썬은 인터프리터 언어이다.",
        "label": "NotNext"
    }
]

print("NSP (Next Sentence Prediction) 예시:")
print("="*60)
for i, ex in enumerate(nsp_examples, 1):
    print(f"\n예시 {i}:")
    print(f"  문장 A: {ex['A']}")
    print(f"  문장 B: {ex['B']}")
    print(f"  레이블: {ex['label']}")

### 사전학습의 의의

```
전통적 방식:        데이터 -> 모델 학습 -> 예측
                   (태스크별로 처음부터)

BERT 방식:         대규모 코퍼스 -> 사전학습 -> 파인튜닝 -> 예측
                   (한 번 학습, 다양한 태스크에 적용)

장점:
1. 적은 데이터로도 높은 성능
2. 다양한 태스크에 범용 적용
3. 학습 시간 대폭 단축
```

---

## 1.3 Hugging Face Transformers 라이브러리

### Hugging Face 소개

**Hugging Face**는 NLP 모델의 민주화를 이끄는 회사로, transformers 라이브러리를 통해 최신 모델을 쉽게 사용할 수 있게 합니다.

```
주요 구성 요소:
1. transformers: 모델 및 토크나이저
2. datasets: 데이터셋 로드 및 처리
3. evaluate: 평가 지표
4. accelerate: 분산 학습 지원
5. Hub: 모델 저장소 (300,000+ 모델)
```

### BERT 기본 모델 구조

```mermaid
flowchart LR
    Input["Input<br/>(batch, seq_len)<br/>token indices"] --> Embedding["Embedding<br/>Token + Position + Segment"]
    Embedding --> Encoder["Transformer<br/>Encoder<br/>(N layers)"]
    Encoder --> LastHidden["Last Hidden State<br/>(batch, seq_len, hidden_size)"]
    Encoder --> Pooler["Pooler<br/>[CLS] token"]
    Pooler --> PoolerOut["Pooler Output<br/>(batch, hidden_size)"]
    
    style Input fill:#ffffff,color:#000000
    style Embedding fill:#ffffff,color:#000000
    style Encoder fill:#ffffff,color:#000000
    style LastHidden fill:#ffffff,color:#000000
    style Pooler fill:#ffffff,color:#000000
    style PoolerOut fill:#ffffff,color:#000000
```

In [ ]:
# Hugging Face transformers 기본 사용법
from transformers import AutoModel, AutoTokenizer, AutoConfig

# 모델 이름 (Hugging Face Hub에서 검색 가능)
model_name = "bert-base-multilingual-cased"

# 설정 확인
config = AutoConfig.from_pretrained(model_name)
print(f"모델: {model_name}")
print(f"\n모델 설정:")
print(f"  - Hidden Size: {config.hidden_size}")
print(f"  - Num Layers: {config.num_hidden_layers}")
print(f"  - Num Attention Heads: {config.num_attention_heads}")
print(f"  - Vocab Size: {config.vocab_size}")
print(f"  - Max Position Embeddings: {config.max_position_embeddings}")

### Auto Classes

Hugging Face의 `Auto` 클래스는 모델 이름만으로 적절한 클래스를 자동 선택합니다.

| Auto Class | 용도 |
|------------|------|
| AutoModel | 기본 모델 |
| AutoModelForSequenceClassification | 텍스트 분류 |
| AutoModelForTokenClassification | 토큰 분류 (NER) |
| AutoModelForQuestionAnswering | 질의응답 |
| AutoTokenizer | 토크나이저 |

---

## 1.4 토크나이저 (Tokenizer)

### 토크나이저란?

텍스트를 모델이 처리할 수 있는 숫자(토큰 ID)로 변환하는 도구입니다.

```
토큰화 과정:
텍스트 -> 토큰 분리 -> 토큰 ID 변환 -> 특수 토큰 추가

"안녕하세요" -> ["안녕", "##하세요"] -> [23456, 78901] -> [101, 23456, 78901, 102]
                                                        [CLS]            [SEP]
```

In [ ]:
# 토크나이저 로드 및 사용
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

# 기본 토큰화
text = "BERT는 자연어 처리의 혁명입니다."
tokens = tokenizer.tokenize(text)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f"원본 텍스트: {text}")
print(f"\n토큰: {tokens}")
print(f"\n토큰 ID: {token_ids}")

In [ ]:
# encode_plus: 실전에서 자주 사용하는 방법
encoded = tokenizer.encode_plus(
    text,
    add_special_tokens=True,    # [CLS], [SEP] 추가
    max_length=20,              # 최대 길이
    padding='max_length',       # 패딩 방식
    truncation=True,            # 잘림 여부
    return_tensors='pt',        # PyTorch 텐서 반환
    return_attention_mask=True  # attention mask 반환
)

print("encode_plus 결과:")
print(f"  input_ids shape: {encoded['input_ids'].shape}")
print(f"  input_ids: {encoded['input_ids'][0].tolist()}")
print(f"  attention_mask: {encoded['attention_mask'][0].tolist()}")
print(f"  token_type_ids: {encoded['token_type_ids'][0].tolist()}")

In [ ]:
# 특수 토큰 확인
print("특수 토큰:")
print(f"  [CLS] ID: {tokenizer.cls_token_id} -> {tokenizer.cls_token}")
print(f"  [SEP] ID: {tokenizer.sep_token_id} -> {tokenizer.sep_token}")
print(f"  [PAD] ID: {tokenizer.pad_token_id} -> {tokenizer.pad_token}")
print(f"  [MASK] ID: {tokenizer.mask_token_id} -> {tokenizer.mask_token}")
print(f"  [UNK] ID: {tokenizer.unk_token_id} -> {tokenizer.unk_token}")

In [ ]:
# 디코딩: 토큰 ID -> 텍스트
decoded = tokenizer.decode(encoded['input_ids'][0], skip_special_tokens=False)
decoded_clean = tokenizer.decode(encoded['input_ids'][0], skip_special_tokens=True)

print(f"디코딩 (특수 토큰 포함): {decoded}")
print(f"디코딩 (특수 토큰 제외): {decoded_clean}")

### Attention Mask

```
Attention Mask:
- 1: 실제 토큰 (모델이 주목)
- 0: 패딩 토큰 (모델이 무시)

예시:
input_ids:      [101, 1234, 5678, 102, 0, 0, 0, 0]
attention_mask: [1,   1,    1,    1,   0, 0, 0, 0]
```

### BERT for Sequence Classification 아키텍처

```mermaid
flowchart LR
    Input["Input<br/>(batch, seq_len)<br/>token indices"] --> BERT["BERT Base<br/>Transformer Encoder<br/>(12 layers)"]
    BERT --> LastHidden["Last Hidden State<br/>(batch, seq_len, 768)"]
    LastHidden --> CLS["[CLS] Token<br/>(batch, 768)"]
    CLS --> Dropout["Dropout"]
    Dropout --> Classifier["Classifier<br/>Linear(768 → num_labels)"]
    Classifier --> Output["Output<br/>(batch, num_labels)<br/>logits"]
    
    style Input fill:#ffffff,color:#000000
    style BERT fill:#ffffff,color:#000000
    style LastHidden fill:#ffffff,color:#000000
    style CLS fill:#ffffff,color:#000000
    style Dropout fill:#ffffff,color:#000000
    style Classifier fill:#ffffff,color:#000000
    style Output fill:#ffffff,color:#000000
```

---

## 1.5 사전학습 모델 로드

### 모델 로드 방법

In [ ]:
# 기본 BERT 모델 로드
from transformers import AutoModel

# 사전학습된 BERT 모델 로드
model = AutoModel.from_pretrained("bert-base-multilingual-cased")

print(f"모델 타입: {type(model).__name__}")
print(f"\n파라미터 수: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# 모델 구조 확인
print("BERT 모델 구조:")
print("="*60)
for name, module in model.named_children():
    print(f"\n{name}:")
    if hasattr(module, 'named_children'):
        for sub_name, sub_module in list(module.named_children())[:3]:
            print(f"  - {sub_name}: {type(sub_module).__name__}")
        if len(list(module.named_children())) > 3:
            print(f"  ... ({len(list(module.named_children()))} 개 레이어)")

In [ ]:
# 모델 순전파 테스트
model.eval()

# 샘플 입력
sample_text = "BERT 모델을 테스트합니다."
inputs = tokenizer(sample_text, return_tensors='pt')

with torch.no_grad():
    outputs = model(**inputs)

print(f"입력 텍스트: {sample_text}")
print(f"\n출력 구조:")
print(f"  - last_hidden_state: {outputs.last_hidden_state.shape}")
print(f"    (batch_size, sequence_length, hidden_size)")
print(f"  - pooler_output: {outputs.pooler_output.shape}")
print(f"    ([CLS] 토큰의 출력)")

### 실무 예시: 문장 임베딩 추출

[CLS] 토큰의 출력(pooler_output)은 문장 전체의 표현으로 사용됩니다.

In [ ]:
# 문장 임베딩 추출 함수
def get_sentence_embedding(text, model, tokenizer):
    """BERT를 사용한 문장 임베딩 추출"""
    model.eval()
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    # pooler_output: [CLS] 토큰의 hidden state
    return outputs.pooler_output

# 유사한 문장과 다른 문장 비교
sentences = [
    "오늘 날씨가 좋다.",
    "날씨가 맑고 화창하다.",
    "파이썬 프로그래밍을 배운다."
]

embeddings = [get_sentence_embedding(s, model, tokenizer) for s in sentences]

# 코사인 유사도 계산
from torch.nn.functional import cosine_similarity

print("문장 간 코사인 유사도:")
print(f"  '{sentences[0]}' vs '{sentences[1]}': {cosine_similarity(embeddings[0], embeddings[1]).item():.4f}")
print(f"  '{sentences[0]}' vs '{sentences[2]}': {cosine_similarity(embeddings[0], embeddings[2]).item():.4f}")

---

# Part 2: 심화

---

## 2.1 다운스트림 태스크 (Downstream Tasks)

### 다운스트림 태스크란?

사전학습된 BERT를 특정 목적에 맞게 활용하는 태스크들입니다.

| 태스크 | 설명 | 출력 형태 |
|--------|------|----------|
| 텍스트 분류 | 문장 -> 카테고리 | [CLS] -> softmax |
| 토큰 분류 (NER) | 각 토큰 -> 라벨 | 모든 토큰 -> softmax |
| 질의응답 (QA) | 질문+문서 -> 답변 위치 | start, end 인덱스 |
| 문장 유사도 | 문장 쌍 -> 유사도 | [CLS] -> sigmoid |

In [ ]:
# 태스크별 모델 헤드 구조 시각화
tasks_info = {
    '텍스트 분류': 'BERT -> [CLS] -> Linear -> Softmax -> 클래스',
    '토큰 분류 (NER)': 'BERT -> 모든 토큰 -> Linear -> Softmax -> 라벨',
    '질의응답 (QA)': 'BERT -> 모든 토큰 -> Linear(2) -> Start/End 위치',
    '문장 유사도': 'BERT -> [CLS] -> Linear -> Sigmoid -> 유사도 점수'
}

print("다운스트림 태스크별 모델 구조:")
print("="*70)
for task, structure in tasks_info.items():
    print(f"\n{task}:")
    print(f"  {structure}")

In [ ]:
# 텍스트 분류용 모델 로드
from transformers import AutoModelForSequenceClassification

# 이진 분류 모델 (num_labels=2)
classification_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=2
)

print("분류 모델 구조:")
print(f"  - BERT: {type(classification_model.bert).__name__}")
print(f"  - Classifier: {classification_model.classifier}")
print(f"\n분류 헤드: Linear(768 -> 2)")

---

## 2.2 Fine-tuning 전략

### Fine-tuning이란?

사전학습된 모델의 가중치를 특정 태스크의 데이터로 미세 조정하는 과정입니다.

```
Fine-tuning 과정:
1. 사전학습 모델 로드 (pre-trained weights)
2. 태스크별 헤드 추가 (classification head)
3. 태스크 데이터로 전체 모델 학습
4. 학습률을 낮게 설정 (보통 2e-5 ~ 5e-5)
5. 적은 에포크 (2~4 epochs)
```

### Fine-tuning 전략 비교

```mermaid
flowchart TB
    subgraph Full["전체 Fine-tuning"]
        FullBERT["BERT<br/>(학습 가능)"] --> FullHead["Classification Head<br/>(학습 가능)"]
        FullHead --> FullOut["Output"]
    end
    
    subgraph Feature["Feature Extraction"]
        FeatureBERT["BERT<br/>(고정, 학습 안함)"] --> FeatureHead["Classification Head<br/>(학습 가능)"]
        FeatureHead --> FeatureOut["Output"]
    end
    
    subgraph Gradual["Gradual Unfreezing"]
        GradualBERT1["BERT 상위 레이어<br/>(학습 가능)"] --> GradualBERT2["BERT 하위 레이어<br/>(고정)"]
        GradualBERT2 --> GradualHead["Classification Head<br/>(학습 가능)"]
        GradualHead --> GradualOut["Output"]
    end
    
    style FullBERT fill:#ffffff,color:#000000
    style FullHead fill:#ffffff,color:#000000
    style FullOut fill:#ffffff,color:#000000
    style FeatureBERT fill:#ffffff,color:#000000
    style FeatureHead fill:#ffffff,color:#000000
    style FeatureOut fill:#ffffff,color:#000000
    style GradualBERT1 fill:#ffffff,color:#000000
    style GradualBERT2 fill:#ffffff,color:#000000
    style GradualHead fill:#ffffff,color:#000000
    style GradualOut fill:#ffffff,color:#000000
```

In [ ]:
# Fine-tuning 전략 비교
strategies = {
    "전체 Fine-tuning": {
        "설명": "BERT + 분류 헤드 전체 학습",
        "장점": "최고 성능",
        "단점": "많은 연산량, 과적합 위험",
        "권장 상황": "데이터가 충분할 때"
    },
    "Feature Extraction": {
        "설명": "BERT 고정, 분류 헤드만 학습",
        "장점": "빠른 학습, 과적합 방지",
        "단점": "성능 제한",
        "권장 상황": "데이터가 적을 때"
    },
    "Gradual Unfreezing": {
        "설명": "상위 레이어부터 점진적으로 학습",
        "장점": "안정적인 학습",
        "단점": "복잡한 구현",
        "권장 상황": "도메인이 다를 때"
    }
}

print("Fine-tuning 전략 비교:")
print("="*70)
for name, info in strategies.items():
    print(f"\n{name}:")
    for key, value in info.items():
        print(f"  - {key}: {value}")

In [ ]:
# Feature Extraction: BERT 가중치 고정
def freeze_bert(model):
    """BERT 레이어 가중치 고정"""
    for param in model.bert.parameters():
        param.requires_grad = False
    print("BERT 가중치가 고정되었습니다.")

def unfreeze_bert(model):
    """BERT 레이어 가중치 해제"""
    for param in model.bert.parameters():
        param.requires_grad = True
    print("BERT 가중치 고정이 해제되었습니다.")

# 학습 가능한 파라미터 수 확인
def count_trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"전체 파라미터: {sum(p.numel() for p in classification_model.parameters()):,}")
print(f"학습 가능 파라미터 (전체): {count_trainable_params(classification_model):,}")

freeze_bert(classification_model)
print(f"학습 가능 파라미터 (BERT 고정): {count_trainable_params(classification_model):,}")

unfreeze_bert(classification_model)
print(f"학습 가능 파라미터 (고정 해제): {count_trainable_params(classification_model):,}")

### Fine-tuning 하이퍼파라미터 권장 값

| 파라미터 | 권장 값 | 설명 |
|---------|---------|------|
| Learning Rate | 2e-5 ~ 5e-5 | 사전학습 가중치를 크게 변경하지 않도록 |
| Epochs | 2 ~ 4 | 적은 에포크로 충분 |
| Batch Size | 16 ~ 32 | GPU 메모리에 따라 조정 |
| Warmup | 전체의 6~10% | 학습 초기 안정화 |
| Weight Decay | 0.01 | 과적합 방지 |

---

## 2.3 한국어 BERT (Multilingual BERT)

### 한국어 지원 모델

| 모델 | 설명 | 특징 |
|------|------|------|
| bert-base-multilingual-cased | 104개 언어 지원 | 범용성 높음 |
| klue/bert-base | 한국어 특화 | KLUE 벤치마크 최적화 |
| monologg/kobert | SKT KoBERT 기반 | 한국어 성능 우수 |
| beomi/kcbert-base | 한국어 댓글 학습 | 비격식체 처리 |

이번 실습에서는 범용적인 `bert-base-multilingual-cased`를 사용합니다.

In [ ]:
# 한국어 토큰화 테스트
korean_texts = [
    "오늘 영화를 봤는데 정말 재미있었어요!",
    "이 영화는 시간 낭비였습니다. 최악이에요.",
    "배우들의 연기가 인상적이었습니다."
]

print("한국어 토큰화 결과:")
print("="*60)
for text in korean_texts:
    tokens = tokenizer.tokenize(text)
    print(f"\n원문: {text}")
    print(f"토큰: {tokens}")

### WordPiece 토큰화

BERT는 **WordPiece** 토큰화를 사용합니다.

```
WordPiece:
- 자주 등장하는 서브워드 단위로 분할
- ##: 단어 중간/끝 부분임을 표시
- OOV(Out-of-Vocabulary) 문제 해결

예시:
"재미있었어요" -> ["재미", "##있", "##었", "##어요"]
```

---

## 2.4 NSMC 한국어 감성 분석 실습

### NSMC (Naver Sentiment Movie Corpus)

네이버 영화 리뷰 데이터셋으로, 한국어 감성 분석의 대표적인 벤치마크입니다.

```
데이터셋 구성:
- 훈련 데이터: 150,000개
- 테스트 데이터: 50,000개
- 라벨: 0 (부정), 1 (긍정)
```

In [ ]:
# NSMC 데이터 로드
# 데이터셋 경로 설정 (로컬 파일 또는 URL)
try:
    # 로컬 파일 시도
    train_df = pd.read_csv("../datasets/text/nsmc/ratings_train.txt", sep="\t")
    test_df = pd.read_csv("../datasets/text/nsmc/ratings_test.txt", sep="\t")
except:
    # URL에서 다운로드
    train_url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt"
    test_url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt"
    train_df = pd.read_csv(train_url, sep="\t")
    test_df = pd.read_csv(test_url, sep="\t")

print(f"훈련 데이터: {len(train_df):,}개")
print(f"테스트 데이터: {len(test_df):,}개")
print(f"\n컬럼: {train_df.columns.tolist()}")
print(f"\n샘플 데이터:")
train_df.head()

In [ ]:
# 라벨 분포 확인
label_counts = train_df['label'].value_counts().reset_index()
label_counts.columns = ['label', 'count']
label_counts['label_name'] = label_counts['label'].map({0: '부정 (0)', 1: '긍정 (1)'})

fig = px.bar(
    label_counts, x='label_name', y='count', 
    color='label_name',
    title='NSMC 라벨 분포',
    labels={'label_name': '라벨', 'count': '개수'}
)
fig.update_layout(template='plotly_white', showlegend=False)
fig.show()

In [ ]:
# 결측치 처리
print(f"결측치 확인:")
print(train_df.isnull().sum())

# 결측치 제거
train_df = train_df.dropna()
test_df = test_df.dropna()

print(f"\n결측치 제거 후: 훈련 {len(train_df):,}개, 테스트 {len(test_df):,}개")

In [ ]:
# 샘플링 (빠른 실습을 위해 데이터 일부만 사용)
# 전체 데이터로 학습하려면 이 셀을 건너뛰세요
SAMPLE_SIZE = 5000  # 훈련용
TEST_SAMPLE_SIZE = 1000  # 테스트용

train_sample = train_df.sample(n=SAMPLE_SIZE, random_state=42).reset_index(drop=True)
test_sample = test_df.sample(n=TEST_SAMPLE_SIZE, random_state=42).reset_index(drop=True)

print(f"샘플링된 데이터:")
print(f"  훈련: {len(train_sample):,}개")
print(f"  테스트: {len(test_sample):,}개")

In [ ]:
# 토크나이저 및 모델 준비
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = "bert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
).to(device)

print(f"모델: {MODEL_NAME}")
print(f"디바이스: {device}")

In [ ]:
# 데이터셋 클래스 정의
from torch.utils.data import Dataset, DataLoader

class NSMCDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        text = str(self.df.iloc[idx]['document'])
        label = self.df.iloc[idx]['label']
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 데이터셋 생성
train_dataset = NSMCDataset(train_sample, tokenizer)
test_dataset = NSMCDataset(test_sample, tokenizer)

# 데이터로더 생성
BATCH_SIZE = 16
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

print(f"데이터셋 준비 완료!")
print(f"  훈련 배치 수: {len(train_loader)}")
print(f"  테스트 배치 수: {len(test_loader)}")

In [ ]:
# 학습 설정
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

# 하이퍼파라미터
EPOCHS = 3
LEARNING_RATE = 2e-5
WARMUP_RATIO = 0.1

# 옵티마이저
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)

# 스케줄러 (warmup + linear decay)
total_steps = len(train_loader) * EPOCHS
warmup_steps = int(total_steps * WARMUP_RATIO)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

print(f"학습 설정:")
print(f"  에포크: {EPOCHS}")
print(f"  학습률: {LEARNING_RATE}")
print(f"  총 스텝: {total_steps}")
print(f"  Warmup 스텝: {warmup_steps}")

In [ ]:
# 학습 함수
def train_epoch(model, dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        total_loss += loss.item()
        
        # 역전파
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        # 정확도 계산
        preds = torch.argmax(outputs.logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    
    return total_loss / len(dataloader), correct / total

# 평가 함수
def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            
            total_loss += outputs.loss.item()
            
            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    return total_loss / len(dataloader), correct / total

In [ ]:
# 학습 실행
print("BERT 파인튜닝 시작!")
print("="*60)

train_losses = []
train_accs = []
eval_losses = []
eval_accs = []

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    
    # 훈련
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, scheduler, device)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # 평가
    eval_loss, eval_acc = evaluate(model, test_loader, device)
    eval_losses.append(eval_loss)
    eval_accs.append(eval_acc)
    
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"  Eval Loss: {eval_loss:.4f}, Eval Acc: {eval_acc:.4f}")

print("\n학습 완료!")

In [ ]:
# 학습 곡선 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=['Loss', 'Accuracy'])

# Loss
fig.add_trace(
    go.Scatter(y=train_losses, name='Train Loss', mode='lines+markers'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(y=eval_losses, name='Eval Loss', mode='lines+markers'),
    row=1, col=1
)

# Accuracy
fig.add_trace(
    go.Scatter(y=train_accs, name='Train Acc', mode='lines+markers'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(y=eval_accs, name='Eval Acc', mode='lines+markers'),
    row=1, col=2
)

fig.update_xaxes(title_text='Epoch')
fig.update_layout(
    title='BERT Fine-tuning 학습 곡선',
    height=400,
    template='plotly_white'
)
fig.show()

In [ ]:
# 새로운 문장 예측
def predict_sentiment(text, model, tokenizer, device):
    """감성 예측 함수"""
    model.eval()
    
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probs = torch.softmax(outputs.logits, dim=1)
        pred = torch.argmax(probs, dim=1).item()
        confidence = probs[0][pred].item()
    
    sentiment = "긍정" if pred == 1 else "부정"
    return sentiment, confidence

# 테스트
test_reviews = [
    "정말 재미있는 영화였어요! 다시 보고 싶네요.",
    "시간 낭비했습니다. 최악의 영화예요.",
    "배우들 연기는 좋았지만 스토리가 아쉬웠어요.",
    "완전 감동받았어요. 눈물이 멈추지 않았습니다.",
    "지루해서 중간에 나왔어요."
]

print("감성 분석 결과:")
print("="*60)
for review in test_reviews:
    sentiment, confidence = predict_sentiment(review, model, tokenizer, device)
    print(f"\n리뷰: {review}")
    print(f"예측: {sentiment} (신뢰도: {confidence:.2%})")

In [ ]:
# 혼동 행렬 생성
from sklearn.metrics import confusion_matrix, classification_report

# 전체 테스트 데이터 예측
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels']
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1).cpu()
        
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.tolist())

# 혼동 행렬
cm = confusion_matrix(all_labels, all_preds)

# 시각화
fig = px.imshow(
    cm,
    labels=dict(x="예측", y="실제", color="개수"),
    x=['부정', '긍정'],
    y=['부정', '긍정'],
    text_auto=True,
    title='BERT 감성 분류 혼동 행렬'
)
fig.update_layout(template='plotly_white')
fig.show()

# 분류 리포트
print("\n분류 리포트:")
print(classification_report(all_labels, all_preds, target_names=['부정', '긍정']))

---

## 실습 퀴즈

**난이도**: (쉬움) ~ (어려움)

---

### Q1. BERT 개념 이해하기

**문제**: BERT가 "Bidirectional"이라고 불리는 이유를 설명하고, 기존 단방향 모델(GPT 등)과의 차이점을 서술하세요.

In [ ]:
# 여기에 답을 작성하세요


### Q2. MLM과 NSP 이해하기

**문제**: BERT의 두 가지 사전학습 과제인 MLM과 NSP의 역할을 각각 한 문장으로 설명하세요.

In [ ]:
# 여기에 답을 작성하세요


### Q3. 토크나이저 사용하기

**문제**: 아래 텍스트를 토크나이저로 변환하고, 결과를 출력하세요.

```python
text = "Hugging Face 라이브러리는 정말 편리합니다!"
```

출력해야 할 것:
1. 토큰 리스트
2. 토큰 ID 리스트
3. attention_mask

In [ ]:
from transformers import AutoTokenizer

text = "Hugging Face 라이브러리는 정말 편리합니다!"

# 여기에 코드를 작성하세요


### Q4. 모델 로드 및 출력 shape

**문제**: `bert-base-multilingual-cased` 모델을 로드하고, 아래 텍스트의 출력 shape을 확인하세요.

```python
text = "딥러닝 모델을 학습합니다."
```

In [ ]:
from transformers import AutoModel, AutoTokenizer

text = "딥러닝 모델을 학습합니다."

# 여기에 코드를 작성하세요


### Q5. 특수 토큰 이해

**문제**: BERT의 특수 토큰 [CLS], [SEP], [PAD], [MASK]의 역할을 각각 설명하세요.

In [ ]:
# 여기에 답을 작성하세요


### Q6. 다운스트림 태스크 이해

**문제**: BERT를 사용한 텍스트 분류에서 왜 [CLS] 토큰의 출력을 사용하는지 설명하세요.

In [ ]:
# 여기에 답을 작성하세요


### Q7. Fine-tuning 전략 비교

**문제**: "전체 Fine-tuning"과 "Feature Extraction" 전략의 차이점을 설명하고, 각각 언제 사용하면 좋은지 서술하세요.

In [ ]:
# 여기에 답을 작성하세요


### Q8. 한국어 토큰화

**문제**: 아래 두 문장을 토크나이저로 변환하고, 토큰화 결과를 비교 분석하세요.

```python
text1 = "자연어 처리"
text2 = "Natural Language Processing"
```

한국어와 영어의 토큰화 차이점을 설명하세요.

In [ ]:
text1 = "자연어 처리"
text2 = "Natural Language Processing"

# 여기에 코드를 작성하세요


### Q9. 감성 분류 모델 개선

**문제**: 본문의 NSMC 감성 분류 코드를 수정하여 다음을 구현하세요.

1. BERT 레이어를 고정(freeze)한 상태로 학습
2. 고정된 상태와 고정하지 않은 상태의 성능 비교

In [ ]:
# 여기에 코드를 작성하세요


### Q10. 전체 파이프라인 구현

**문제**: Hugging Face의 `pipeline` API를 사용하여 감성 분석을 수행하는 코드를 작성하세요.

테스트 리뷰:
```python
reviews = [
    "이 제품 정말 좋아요! 강추합니다.",
    "배송이 너무 늦고 포장이 엉망이었어요.",
    "가격 대비 괜찮은 것 같아요."
]
```

In [ ]:
from transformers import pipeline

reviews = [
    "이 제품 정말 좋아요! 강추합니다.",
    "배송이 너무 늦고 포장이 엉망이었어요.",
    "가격 대비 괜찮은 것 같아요."
]

# 여기에 코드를 작성하세요


---

## 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 내용 | 실무 활용 |
|-----|----------|----------|
| BERT | 양방향 Transformer Encoder | 범용 언어 이해 모델 |
| MLM | 마스킹된 토큰 예측 | 문맥적 단어 이해 |
| NSP | 다음 문장 예측 | 문장 관계 이해 |
| Hugging Face | transformers 라이브러리 | 3줄로 최신 모델 사용 |
| Tokenizer | 텍스트 -> 토큰 ID | 모델 입력 준비 |

### Part 2: 심화 핵심 요약

| 개념 | 핵심 내용 | 언제 사용? |
|-----|----------|----------|
| 다운스트림 태스크 | 분류, NER, QA 등 | 특정 목적의 NLP |
| Fine-tuning | 사전학습 모델 미세조정 | 적은 데이터로 고성능 |
| Feature Extraction | BERT 고정, 헤드만 학습 | 데이터 부족 시 |
| 한국어 BERT | mBERT, KoBERT 등 | 한국어 NLP 태스크 |

### BERT 활용 가이드

```
1. 모델 선택:
   - 범용: bert-base-multilingual-cased
   - 한국어 특화: klue/bert-base, monologg/kobert

2. Fine-tuning 전략:
   - 데이터 충분: 전체 Fine-tuning
   - 데이터 부족: Feature Extraction
   - 도메인 차이: Gradual Unfreezing

3. 하이퍼파라미터:
   - Learning Rate: 2e-5 ~ 5e-5
   - Epochs: 2 ~ 4
   - Batch Size: 16 ~ 32
```

### 실무 팁

1. **토크나이저와 모델은 같은 것 사용**: 반드시 동일한 pretrained 모델명 사용
2. **max_length 설정**: 데이터에 맞게 조정 (메모리 vs 정보량)
3. **Warmup 사용**: 학습 초기 불안정 방지
4. **Gradient Clipping**: 기울기 폭발 방지 (max_norm=1.0)
5. **조기 종료**: 검증 성능 기반 Early Stopping 권장